# Spark SQL

## Objetivos de Aprendizaje
- Ejecutar consultas SQL sobre DataFrames
- Usar tablas temporales y vistas
- Aplicar funciones SQL avanzadas
- Combinar SQL con la API de DataFrames

## Prerequisitos
- `06_spark_processing/02_dataframes_api.ipynb`
- Conocimientos básicos de SQL

## Tiempo Estimado
⏱️ 45 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9 y 11: Spark SQL es similar a Amazon Athena

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import date

spark = SparkSession.builder \
    .appName("SparkSQL") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(f"Spark SQL {spark.version} listo")

---
# === SECCIÓN 1 ===
## 1. Crear Datos de Ejemplo

### Explicación Conceptual
Crearemos tablas de ejemplo que simulan un sistema de e-commerce para practicar SQL.

In [ ]:
# Tabla de productos
productos_data = [
    (1, "Laptop HP", "Electronica", 15000.00),
    (2, "Mouse Logitech", "Electronica", 350.00),
    (3, "Monitor Dell", "Electronica", 8000.00),
    (4, "Silla Ergonomica", "Muebles", 4500.00),
    (5, "Escritorio", "Muebles", 6000.00),
    (6, "Teclado Mecanico", "Electronica", 1200.00),
    (7, "Webcam HD", "Electronica", 800.00),
    (8, "Lampara LED", "Muebles", 450.00)
]
df_productos = spark.createDataFrame(
    productos_data, 
    ["producto_id", "nombre", "categoria", "precio"]
)

# Tabla de clientes
clientes_data = [
    ("C001", "Ana Garcia", "CDMX", "Premium"),
    ("C002", "Carlos Lopez", "Guadalajara", "Regular"),
    ("C003", "Maria Rodriguez", "Monterrey", "Premium"),
    ("C004", "Juan Martinez", "CDMX", "Regular"),
    ("C005", "Laura Sanchez", "Puebla", "Premium")
]
df_clientes = spark.createDataFrame(
    clientes_data,
    ["cliente_id", "nombre", "ciudad", "tipo"]
)

# Tabla de ventas
ventas_data = [
    ("V001", "C001", 1, 2, date(2024, 1, 15)),
    ("V002", "C001", 2, 3, date(2024, 1, 15)),
    ("V003", "C002", 3, 1, date(2024, 1, 16)),
    ("V004", "C003", 1, 1, date(2024, 1, 17)),
    ("V005", "C002", 6, 2, date(2024, 1, 17)),
    ("V006", "C004", 4, 1, date(2024, 1, 18)),
    ("V007", "C001", 7, 2, date(2024, 1, 18)),
    ("V008", "C005", 5, 1, date(2024, 1, 19)),
    ("V009", "C003", 2, 5, date(2024, 1, 19)),
    ("V010", "C002", 8, 3, date(2024, 1, 20))
]
df_ventas = spark.createDataFrame(
    ventas_data,
    ["venta_id", "cliente_id", "producto_id", "cantidad", "fecha"]
)

print("Tablas creadas:")
print(f"  Productos: {df_productos.count()} filas")
print(f"  Clientes: {df_clientes.count()} filas")
print(f"  Ventas: {df_ventas.count()} filas")

In [ ]:
# Registrar DataFrames como tablas SQL temporales
# createOrReplaceTempView() crea una vista accesible via SQL

df_productos.createOrReplaceTempView("productos")
df_clientes.createOrReplaceTempView("clientes")
df_ventas.createOrReplaceTempView("ventas")

print("Tablas registradas para SQL:")
spark.sql("SHOW TABLES").show()

---
# === SECCIÓN 2 ===
## 2. Consultas SELECT Básicas

### Explicación Conceptual
Spark SQL soporta SQL estándar ANSI. Puedes usar `spark.sql()` para ejecutar cualquier consulta SQL y obtener un DataFrame como resultado.

In [ ]:
# SELECT basico
print("Todos los productos:")
spark.sql("SELECT * FROM productos").show()

# SELECT con columnas especificas
print("Solo nombre y precio:")
spark.sql("""
    SELECT nombre, precio 
    FROM productos
""").show()

In [ ]:
# WHERE para filtrar
print("Productos de Electronica con precio > 1000:")
spark.sql("""
    SELECT nombre, categoria, precio
    FROM productos
    WHERE categoria = 'Electronica'
      AND precio > 1000
    ORDER BY precio DESC
""").show()

In [ ]:
# Columnas calculadas y alias
print("Productos con precio con IVA:")
spark.sql("""
    SELECT 
        nombre,
        precio AS precio_sin_iva,
        precio * 1.16 AS precio_con_iva,
        UPPER(categoria) AS categoria_mayusculas
    FROM productos
    ORDER BY precio DESC
    LIMIT 5
""").show()

---
# === SECCIÓN 3 ===
## 3. Agregaciones con GROUP BY

### Explicación Conceptual
GROUP BY agrupa filas y permite aplicar funciones de agregación como SUM, COUNT, AVG, etc.

In [ ]:
# Agregaciones basicas
print("Resumen de productos por categoria:")
spark.sql("""
    SELECT 
        categoria,
        COUNT(*) AS num_productos,
        ROUND(AVG(precio), 2) AS precio_promedio,
        MIN(precio) AS precio_minimo,
        MAX(precio) AS precio_maximo,
        SUM(precio) AS suma_precios
    FROM productos
    GROUP BY categoria
""").show()

In [ ]:
# Ventas por cliente
print("Total de ventas por cliente:")
spark.sql("""
    SELECT 
        cliente_id,
        COUNT(*) AS num_compras,
        SUM(cantidad) AS unidades_totales
    FROM ventas
    GROUP BY cliente_id
    ORDER BY num_compras DESC
""").show()

In [ ]:
# HAVING para filtrar despues de agrupar
print("Clientes con mas de 2 compras:")
spark.sql("""
    SELECT 
        cliente_id,
        COUNT(*) AS num_compras
    FROM ventas
    GROUP BY cliente_id
    HAVING COUNT(*) > 2
""").show()

---
# === SECCIÓN 4 ===
## 4. JOINs entre Tablas

### Explicación Conceptual
Los JOINs combinan datos de múltiples tablas basándose en columnas relacionadas.

**Tipos de JOIN:**
- **INNER JOIN**: Solo filas que coinciden en ambas tablas
- **LEFT JOIN**: Todas las filas de la izquierda + coincidencias
- **RIGHT JOIN**: Todas las filas de la derecha + coincidencias
- **FULL OUTER JOIN**: Todas las filas de ambas tablas

In [ ]:
# INNER JOIN: Ventas con informacion de productos
print("Detalle de ventas con productos:")
spark.sql("""
    SELECT 
        v.venta_id,
        p.nombre AS producto,
        v.cantidad,
        p.precio,
        v.cantidad * p.precio AS total
    FROM ventas v
    INNER JOIN productos p ON v.producto_id = p.producto_id
    ORDER BY total DESC
""").show()

In [ ]:
# JOIN de 3 tablas: Ventas completas
print("Reporte completo de ventas:")
spark.sql("""
    SELECT 
        v.venta_id,
        v.fecha,
        c.nombre AS cliente,
        c.ciudad,
        p.nombre AS producto,
        p.categoria,
        v.cantidad,
        p.precio,
        v.cantidad * p.precio AS total
    FROM ventas v
    JOIN clientes c ON v.cliente_id = c.cliente_id
    JOIN productos p ON v.producto_id = p.producto_id
    ORDER BY v.fecha, v.venta_id
""").show(truncate=False)

In [ ]:
# LEFT JOIN: Todos los clientes, incluso sin ventas
# Primero agregamos un cliente sin ventas
df_clientes_extra = spark.createDataFrame(
    [("C006", "Pedro Nuevo", "Tijuana", "Regular")],
    ["cliente_id", "nombre", "ciudad", "tipo"]
)
df_clientes.union(df_clientes_extra).createOrReplaceTempView("clientes")

print("Todos los clientes con sus ventas (incluso sin ventas):")
spark.sql("""
    SELECT 
        c.cliente_id,
        c.nombre,
        COUNT(v.venta_id) AS num_ventas,
        COALESCE(SUM(v.cantidad), 0) AS unidades_totales
    FROM clientes c
    LEFT JOIN ventas v ON c.cliente_id = v.cliente_id
    GROUP BY c.cliente_id, c.nombre
    ORDER BY num_ventas DESC
""").show()

---
# === SECCIÓN 5 ===
## 5. Subconsultas y CTEs

### Explicación Conceptual
Las **subconsultas** son consultas dentro de otras consultas. Los **CTEs (Common Table Expressions)** con `WITH` hacen las consultas más legibles.

In [ ]:
# Subconsulta en WHERE
print("Productos con precio mayor al promedio:")
spark.sql("""
    SELECT nombre, precio
    FROM productos
    WHERE precio > (SELECT AVG(precio) FROM productos)
    ORDER BY precio DESC
""").show()

In [ ]:
# Subconsulta en FROM (tabla derivada)
print("Categoria con mayor promedio de precio:")
spark.sql("""
    SELECT *
    FROM (
        SELECT 
            categoria,
            ROUND(AVG(precio), 2) AS precio_promedio,
            COUNT(*) AS num_productos
        FROM productos
        GROUP BY categoria
    ) subquery
    WHERE precio_promedio > 2000
    ORDER BY precio_promedio DESC
""").show()

In [ ]:
# CTE (WITH) - mas legible para consultas complejas
print("Analisis de clientes con CTE:")
spark.sql("""
    WITH ventas_cliente AS (
        SELECT 
            cliente_id,
            COUNT(*) AS num_compras,
            SUM(cantidad) AS unidades
        FROM ventas
        GROUP BY cliente_id
    ),
    promedio_compras AS (
        SELECT AVG(num_compras) AS promedio
        FROM ventas_cliente
    )
    SELECT 
        c.nombre,
        c.tipo,
        vc.num_compras,
        vc.unidades,
        CASE 
            WHEN vc.num_compras > p.promedio THEN 'Por encima del promedio'
            ELSE 'Por debajo del promedio'
        END AS clasificacion
    FROM ventas_cliente vc
    JOIN clientes c ON vc.cliente_id = c.cliente_id
    CROSS JOIN promedio_compras p
    ORDER BY vc.num_compras DESC
""").show(truncate=False)

---
# === SECCIÓN 6 ===
## 6. Funciones de Ventana (Window Functions)

### Explicación Conceptual
Las funciones de ventana calculan valores sobre un conjunto de filas relacionadas, sin agrupar (no reducen el número de filas).

In [ ]:
# ROW_NUMBER: Numerar filas dentro de grupos
print("Productos numerados por categoria (por precio):")
spark.sql("""
    SELECT 
        categoria,
        nombre,
        precio,
        ROW_NUMBER() OVER (PARTITION BY categoria ORDER BY precio DESC) AS ranking
    FROM productos
""").show()

In [ ]:
# RANK y DENSE_RANK
print("Comparacion de rankings:")
spark.sql("""
    SELECT 
        nombre,
        precio,
        ROW_NUMBER() OVER (ORDER BY precio DESC) AS row_num,
        RANK() OVER (ORDER BY precio DESC) AS rank,
        DENSE_RANK() OVER (ORDER BY precio DESC) AS dense_rank
    FROM productos
""").show()

In [ ]:
# Acumulados y promedios moviles
print("Ventas con acumulado por fecha:")
spark.sql("""
    SELECT 
        fecha,
        venta_id,
        cantidad,
        SUM(cantidad) OVER (ORDER BY fecha ROWS UNBOUNDED PRECEDING) AS cantidad_acumulada,
        AVG(cantidad) OVER (ORDER BY fecha ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS promedio_movil_3
    FROM ventas
    ORDER BY fecha, venta_id
""").show()

In [ ]:
# LAG y LEAD: Valores anteriores y siguientes
print("Comparar con venta anterior:")
spark.sql("""
    SELECT 
        venta_id,
        fecha,
        cantidad,
        LAG(cantidad, 1) OVER (ORDER BY fecha, venta_id) AS cantidad_anterior,
        LEAD(cantidad, 1) OVER (ORDER BY fecha, venta_id) AS cantidad_siguiente,
        cantidad - LAG(cantidad, 1) OVER (ORDER BY fecha, venta_id) AS diferencia
    FROM ventas
""").show()

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio SQL.1: Reporte de Ventas

Crea un reporte que muestre:
- Fecha
- Total de ventas del día (suma de cantidad × precio)
- Número de transacciones
- Producto más vendido del día

Ordena por fecha.

In [ ]:
# TODO: Escribe tu consulta SQL


### ✅ Solución Ejercicio SQL.1

In [ ]:
spark.sql("""
    WITH ventas_detalle AS (
        SELECT 
            v.fecha,
            v.venta_id,
            p.nombre AS producto,
            v.cantidad,
            v.cantidad * p.precio AS total_venta
        FROM ventas v
        JOIN productos p ON v.producto_id = p.producto_id
    ),
    producto_top AS (
        SELECT 
            fecha,
            producto,
            SUM(cantidad) AS unidades,
            ROW_NUMBER() OVER (PARTITION BY fecha ORDER BY SUM(cantidad) DESC) AS rn
        FROM ventas_detalle
        GROUP BY fecha, producto
    )
    SELECT 
        vd.fecha,
        ROUND(SUM(vd.total_venta), 2) AS venta_total_dia,
        COUNT(DISTINCT vd.venta_id) AS num_transacciones,
        MAX(pt.producto) AS producto_mas_vendido
    FROM ventas_detalle vd
    JOIN producto_top pt ON vd.fecha = pt.fecha AND pt.rn = 1
    GROUP BY vd.fecha, pt.producto
    ORDER BY vd.fecha
""").show(truncate=False)

### 🎯 Ejercicio SQL.2: Ranking de Clientes

Crea una consulta que:
1. Calcule el gasto total de cada cliente
2. Asigne un ranking por gasto
3. Calcule el porcentaje del total de ventas que representa cada cliente
4. Solo muestre los top 3

In [ ]:
# TODO: Escribe tu consulta SQL


### ✅ Solución Ejercicio SQL.2

In [ ]:
spark.sql("""
    WITH gasto_cliente AS (
        SELECT 
            c.cliente_id,
            c.nombre,
            c.tipo,
            SUM(v.cantidad * p.precio) AS gasto_total
        FROM clientes c
        JOIN ventas v ON c.cliente_id = v.cliente_id
        JOIN productos p ON v.producto_id = p.producto_id
        GROUP BY c.cliente_id, c.nombre, c.tipo
    ),
    total_ventas AS (
        SELECT SUM(gasto_total) AS total FROM gasto_cliente
    )
    SELECT 
        gc.nombre,
        gc.tipo,
        gc.gasto_total,
        RANK() OVER (ORDER BY gc.gasto_total DESC) AS ranking,
        ROUND(gc.gasto_total / tv.total * 100, 2) AS porcentaje_total
    FROM gasto_cliente gc
    CROSS JOIN total_ventas tv
    ORDER BY ranking
    LIMIT 3
""").show()

### 🎯 Ejercicio SQL.3: Análisis de Tendencias

Usando funciones de ventana, muestra para cada venta:
- La venta actual
- El total acumulado hasta esa fecha
- La diferencia con la venta anterior
- Si la venta fue mayor, menor o igual a la anterior

In [ ]:
# TODO: Escribe tu consulta SQL


### ✅ Solución Ejercicio SQL.3

In [ ]:
spark.sql("""
    WITH ventas_valor AS (
        SELECT 
            v.venta_id,
            v.fecha,
            v.cantidad * p.precio AS monto
        FROM ventas v
        JOIN productos p ON v.producto_id = p.producto_id
    )
    SELECT 
        venta_id,
        fecha,
        monto AS venta_actual,
        SUM(monto) OVER (ORDER BY fecha, venta_id ROWS UNBOUNDED PRECEDING) AS acumulado,
        LAG(monto) OVER (ORDER BY fecha, venta_id) AS venta_anterior,
        monto - LAG(monto) OVER (ORDER BY fecha, venta_id) AS diferencia,
        CASE 
            WHEN monto > LAG(monto) OVER (ORDER BY fecha, venta_id) THEN 'MAYOR'
            WHEN monto < LAG(monto) OVER (ORDER BY fecha, venta_id) THEN 'MENOR'
            WHEN monto = LAG(monto) OVER (ORDER BY fecha, venta_id) THEN 'IGUAL'
            ELSE 'PRIMERA'
        END AS tendencia
    FROM ventas_valor
    ORDER BY fecha, venta_id
""").show()

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Vistas temporales**: `createOrReplaceTempView()` registra DataFrames para SQL
- **Agregaciones**: `GROUP BY` con `COUNT`, `SUM`, `AVG`, `MIN`, `MAX`
- **JOINs**: `INNER`, `LEFT`, `RIGHT`, `FULL OUTER` para combinar tablas
- **Subconsultas**: Consultas anidadas en `WHERE`, `FROM`, o `SELECT`
- **CTEs**: `WITH` para consultas más legibles
- **Window Functions**: `ROW_NUMBER`, `RANK`, `LAG`, `LEAD`, agregados con `OVER`

### Conexión con AWS
- **Amazon Athena**: SQL sobre datos en S3, sintaxis muy similar
- **Amazon Redshift**: Data warehouse con SQL compatible
- **AWS Glue**: Soporta Spark SQL para ETL

### Siguiente Paso
Continúa con: `04_joins_aggregations.ipynb` para joins y agregaciones avanzadas